In [ ]:
import os
import yaml

def fix_data_yaml():
    """Fix the paths in data.yaml"""
    
    # Get the current working directory
    current_dir = os.getcwd()
    
    # Create the correct absolute paths
    file_classes = 'data/labels/classes.txt'
    names = [n for n in open(file_classes).read().split('\n') if n != '']
    yaml_content = {
        'train': os.path.join(current_dir, 'data/images/train'),
        'val': os.path.join(current_dir, 'data/images/val'),
        'nc': len(names),  # number of classes
        'names': names  # class names
    }
    
    # Save the updated data.yaml
    yaml_path = 'data/data.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f)
    
    print("Updated data.yaml with absolute paths:")
    print(f"train: {yaml_content['train']}")
    print(f"val: {yaml_content['val']}")

# if __name__ == "__main__":
fix_data_yaml()

In [1]:
from ultralytics import YOLO
import torch
import os
from datetime import datetime

def train_custom_model(
    data_yaml='data/data.yaml',
    model_size='x',  # n (nano), s (small), m (medium), l (large), x (xlarge)
    epochs=100,
    imgsz=640,
    batch_size=16,
    project='runs/train',
    name=None
):
    """
    Train a custom YOLO11 model
    """
    # Create run name with timestamp if not provided
    if name is None:
        name = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Print system info
    print("\n🖥️ System Information:")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    # Initialize model
    model = YOLO(f'yolo11{model_size}.pt')
    
    print(f"\n🚀 Starting training with following configuration:")
    print(f"Model: YOLO11{model_size}")
    print(f"Epochs: {epochs}")
    print(f"Image size: {imgsz}")
    print(f"Batch size: {batch_size}")
    print(f"Data config: {data_yaml}")
    
    # Train the model
    try:
        results = model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch_size,
            project=project,
            name=name,
            pretrained=True,
            optimizer='Adam',  # Adam optimizer
            patience=50,       # Early stopping patience
            save=True,        # Save checkpoints
            device='0' if torch.cuda.is_available() else 'cpu',  # Use GPU if available
            verbose=True
        )
        
        print("\n✅ Training completed successfully!")
        print(f"Model saved in: {os.path.join(project, name)}")
        
        return model, results
        
    except Exception as e:
        print(f"\n❌ Training error: {str(e)}")
        return None, None

# if __name__ == "__main__":
# Train model with default settings
model, results = train_custom_model(
    model_size='l',    # Start with nano model for quick training
    epochs=100,        # Number of epochs
    imgsz=640,         # Image size
    batch_size=16,      # Batch size (adjust based on your GPU memory)
    name='ayana_tram_stop_detection'  # Run name
)


🖥️ System Information:
CUDA available: False

🚀 Starting training with following configuration:
Model: YOLO11l
Epochs: 100
Image size: 640
Batch size: 16
Data config: data/data.yaml
New https://pypi.org/project/ultralytics/8.3.49 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.8.18 torch-2.2.0 CPU (Apple M2 Pro)
engine/trainer: task=detect, mode=train, model=yolo11l.pt, data=data/data.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=runs/train, name=ayana_tram_stop_detection2, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, p

train: Scanning /Users/fchrulk/AHM/Production/ayana-tram-detection-cctv/build_custom_model/data/labels/train... 144 images, 0 backgrounds, 0 corrupt: 100%|██████████| 144/144 [00:00<00:00, 2557.00it/s]

train: New cache created: /Users/fchrulk/AHM/Production/ayana-tram-detection-cctv/build_custom_model/data/labels/train.cache



val: Scanning /Users/fchrulk/AHM/Production/ayana-tram-detection-cctv/build_custom_model/data/labels/val... 37 images, 0 backgrounds, 0 corrupt: 100%|██████████| 37/37 [00:00<00:00, 2945.33it/s]

val: New cache created: /Users/fchrulk/AHM/Production/ayana-tram-detection-cctv/build_custom_model/data/labels/val.cache


...

    100/100         0G     0.5531     0.3906     0.9463         21        640: 100%|██████████| 9/9 [06:05<00:00, 40.64s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:19<00:00,  9.78s/it]

                   all         37         63      0.928        0.9      0.922      0.756



100 epochs completed in 11.668 hours.
Optimizer stripped from runs/train/ayana_tram_stop_detection2/weights/last.pt, 51.2MB
Optimizer stripped from runs/train/ayana_tram_stop_detection2/weights/best.pt, 51.2MB

Validating runs/train/ayana_tram_stop_detection2/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.8.18 torch-2.2.0 CPU (Apple M2 Pro)
YOLO11l summary (fused): 464 layers, 25,281,625 parameters, 0 gradients, 86.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:19<00:00,  9.50s/it]


                   all         37         63      0.976      0.859      0.939      0.772
            ayana_tram         37         44      0.976      0.935      0.993      0.806
                   car         14         14      0.966      0.643      0.829      0.633
           ayana_buggy          5          5      0.987          1      0.995      0.876
Speed: 0.5ms preprocess, 499.4ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/train/ayana_tram_stop_detection2

✅ Training completed successfully!
Model saved in: runs/train/ayana_tram_stop_detection


In [ ]:
# First train result: 20241206_213249